In [1]:
import time
import rasterio
import numpy as np
import math
from osgeo import gdal
import glob
import geopandas as gpd
import pandas as pd
from pathlib import Path

## 1. 수목 포인트 픽셀 값 가져오기 = raster value sampling

In [2]:
# raster값을 가져올 포인트 데이터에 geometry값 넣기
gdf = gpd.GeoDataFrame.from_file("tree_points/manup_treepoint.shp")
gdf.head()

,OBJECTID,Lat,Long,GenusSpeci,subclass,geometry
0,1401049,40.804813,-73.950177,Acer campestre - hedge maple,Rosidae,POINT (-73.95018 40.80481)
1,270934,40.809214,-73.951732,Zelkova serrata - Japanese zelkova,Hamamelididae,POINT (-73.95173 40.80921)
2,1401056,40.804494,-73.949422,Prunus serrulata 'Not Green leaf' - 'Not Green...,Rosidae,POINT (-73.94942 40.80449)
3,1401045,40.805001,-73.950625,Zelkova serrata - Japanese zelkova,Hamamelididae,POINT (-73.95062 40.80500)
4,1401047,40.804900,-73.950385,Quercus velutina - black oak,Hamamelididae,POINT (-73.95038 40.80490)


In [3]:
print(type(gdf))
print(gdf.dtypes)
gdf.tail()

<class 'geopandas.geodataframe.GeoDataFrame'>
OBJECTID         int64
Lat            float64
Long           float64
GenusSpeci      object
subclass        object
geometry      geometry
dtype: object


,OBJECTID,Lat,Long,GenusSpeci,subclass,geometry
28268,5273853,40.793366,-73.934973,Platanus x acerifolia - London planetree,Hamamelididae,POINT (-73.93497 40.79337)
28269,5605087,40.814567,-73.924720,Quercus palustris - pin oak,Hamamelididae,POINT (-73.92472 40.81457)
28270,5650334,40.813503,-73.928576,Platanus x acerifolia - London planetree,Hamamelididae,POINT (-73.92858 40.81350)
28271,5605086,40.814642,-73.924666,Quercus palustris - pin oak,Hamamelididae,POINT (-73.92467 40.81464)
28272,5273852,40.793318,-73.934868,Platanus x acerifolia - London planetree,Hamamelididae,POINT (-73.93487 40.79332)


In [11]:
# 포인트의 좌표를 리스트로 추출
coord_list = [(x,y) for x,y in zip(gdf['geometry'].x , gdf['geometry'].y)]
print(type(coord_list))
print(coord_list[0:5])

<class 'list'>
[(-73.9501773, 40.80481272), (-73.95173153, 40.80921367), (-73.94942213, 40.80449428), (-73.95062453, 40.80500131), (-73.9503848, 40.80490022)]


In [5]:
#NDVI point sampling
NDVI_file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/NDVI.tif'
NDVI_src = rasterio.open(NDVI_file) # 파일 읽기
gdf[Path(NDVI_file).stem] = [x for x in NDVI_src.sample(coord_list)] # 포인트 샘플링
gdf[Path(NDVI_file).stem] = gdf[Path(NDVI_file).stem].astype('float64')
gdf.head()

# # SBI point sampling
# SBI_file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/SBI.tif'
# SBI_src = rasterio.open(SBI_file) # 파일 읽기
# gdf[Path(SBI_file).stem] = [x for x in SBI_src.sample(coord_list)] # 포인트 샘플링
# gdf[Path(SBI_file).stem] = gdf[Path(SBI_file).stem].astype('float64')

# # GVI point sampling
# GVI_file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/GVI.tif'
# GVI_src = rasterio.open(GVI_file) # 파일 읽기
# gdf[Path(GVI_file).stem] = [x for x in GVI_src.sample(coord_list)] # 포인트 샘플링
# gdf[Path(GVI_file).stem] = gdf[Path(GVI_file).stem].astype('float64')

# # YVI point sampling
# YVI_file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/YVI.tif'
# YVI_src = rasterio.open(YVI_file) # 파일 읽기
# gdf[Path(YVI_file).stem] = [x for x in YVI_src.sample(coord_list)] # 포인트 샘플링
# gdf[Path(YVI_file).stem] = gdf[Path(YVI_file).stem].astype('float64')

# # WBI point sampling
# WBI_file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/WBI.tif'
# WBI_src = rasterio.open(WBI_file) # 파일 읽기
# gdf[Path(WBI_file).stem] = [x for x in WBI_src.sample(coord_list)] # 포인트 샘플링
# gdf[Path(WBI_file).stem] = gdf[Path(WBI_file).stem].astype('float64')
# gdf.head()

,OBJECTID,Lat,Long,GenusSpeci,subclass,geometry,NDVI
0,1401049,40.804813,-73.950177,Acer campestre - hedge maple,Rosidae,POINT (-73.95018 40.80481),0.0
1,270934,40.809214,-73.951732,Zelkova serrata - Japanese zelkova,Hamamelididae,POINT (-73.95173 40.80921),0.0
2,1401056,40.804494,-73.949422,Prunus serrulata 'Not Green leaf' - 'Not Green...,Rosidae,POINT (-73.94942 40.80449),0.0
3,1401045,40.805001,-73.950625,Zelkova serrata - Japanese zelkova,Hamamelididae,POINT (-73.95062 40.80500),0.0
4,1401047,40.804900,-73.950385,Quercus velutina - black oak,Hamamelididae,POINT (-73.95038 40.80490),0.0


## 2. 픽셀값이 0.2 이하인 수목 포인트 버리기

In [6]:
gdf0 = gdf.drop(gdf[gdf['NDVI'] < 0.2].index)
gdf0.head()

,OBJECTID,Lat,Long,GenusSpeci,subclass,geometry,NDVI


## 3. 남은 픽셀 기준 3by3 픽셀 가져오기

In [7]:
# 남은 포인트 픽셀값을 가지고 있는 .csv파일은 'point'
# 근데 여기서 그 포인트 주변값을 가져오려면 본영상의 픽셀값을 가지고 있는 배열을 불러와야 함....


In [8]:
coord_list1= [(x-0.00000599299783809013749338974087784241,y+0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list2= [(x,y+0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list3= [(x+0.00000599299783809013749338974087784241,y+0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list4= [(x+0.00000599299783809013749338974087784241,y) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list5= [(x+0.00000599299783809013749338974087784241,y-0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list6= [(x,y-0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list7= [(x-0.00000599299783809013749338974087784241,y-0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list8= [(x-0.00000599299783809013749338974087784241,y) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]

In [9]:
ndf1=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ndf2=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ndf3=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ndf4=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ndf5=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ndf6=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ndf7=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ndf8=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)

In [10]:
# def pointsampling(file, src, feature):
#     for i in range(1, 9):
#         ndf[Path(file).stem] = [x for x in src.sample(coord_list1)] # 포인트 샘플링
#         ndf[Path(file).stem] = ndf1[Path(file).stem].astype('float64')
#         ndf1.rename(columns={feature: feautre+'_'+i}, inplace=True)
#         ndf1.head()
#         print(len(ndf1.index))

In [11]:
file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/NDVI.tif'
src = rasterio.open(file) # 파일 읽기
# NDVI 0-8 numbering
#1
ndf1[Path(file).stem] = [x for x in src.sample(coord_list1)] # 포인트 샘플링
ndf1[Path(file).stem] = ndf1[Path(file).stem].astype('float64')
ndf1.rename(columns={'NDVI': 'NDVI_1'}, inplace=True)
ndf1.head()
print(len(ndf1.index))

#2
ndf2[Path(file).stem] = [x for x in src.sample(coord_list2)] # 포인트 샘플링
ndf2[Path(file).stem] = ndf2[Path(file).stem].astype('float64')
ndf2.rename(columns={'NDVI': 'NDVI_2'}, inplace=True)
ndf2.head()
print(len(ndf2.index))

#3
ndf3[Path(file).stem] = [x for x in src.sample(coord_list3)] # 포인트 샘플링
ndf3[Path(file).stem] = ndf3[Path(file).stem].astype('float64')
ndf3.rename(columns={'NDVI': 'NDVI_3'}, inplace=True)
ndf3.head()
print(len(ndf3.index))

#4
ndf4[Path(file).stem] = [x for x in src.sample(coord_list4)] # 포인트 샘플링
ndf4[Path(file).stem] = ndf4[Path(file).stem].astype('float64')
ndf4.rename(columns={'NDVI': 'NDVI_4'}, inplace=True)
ndf4.head()
print(len(ndf4.index))

#5
ndf5[Path(file).stem] = [x for x in src.sample(coord_list5)] # 포인트 샘플링
ndf5[Path(file).stem] = ndf5[Path(file).stem].astype('float64')
ndf5.rename(columns={'NDVI': 'NDVI_5'}, inplace=True)
ndf5.head()
print(len(ndf5.index))

#6
ndf6[Path(file).stem] = [x for x in src.sample(coord_list6)] # 포인트 샘플링
ndf6[Path(file).stem] = ndf6[Path(file).stem].astype('float64')
ndf6.rename(columns={'NDVI': 'NDVI_6'}, inplace=True)
ndf6.head()
print(len(ndf6.index))

#7
ndf7[Path(file).stem] = [x for x in src.sample(coord_list7)] # 포인트 샘플링
ndf7[Path(file).stem] = ndf7[Path(file).stem].astype('float64')
ndf7.rename(columns={'NDVI': 'NDVI_7'}, inplace=True)
ndf7.head()
print(len(ndf7.index))

#8
ndf8[Path(file).stem] = [x for x in src.sample(coord_list8)] # 포인트 샘플링
ndf8[Path(file).stem] = ndf8[Path(file).stem].astype('float64')
ndf8.rename(columns={'NDVI': 'NDVI_8'}, inplace=True)
ndf8.head()
print(len(ndf8.index))

0
0
0
0
0
0
0
0


In [12]:
#1
gdf0['NDVI_1'] = ndf1['NDVI_1']
gdf0['NDVI_1'] = np.where(
    gdf0['NDVI_1'] < 0.2, 
    0,
    gdf0['NDVI_1'])

#2
gdf0['NDVI_2'] = ndf2['NDVI_2']
gdf0['NDVI_2'] = np.where(
    gdf0['NDVI_2'] < 0.2, 
    0, 
    gdf0['NDVI_2'])

#3
gdf0['NDVI_3'] = ndf3['NDVI_3']
gdf0['NDVI_3'] = np.where(
    gdf0['NDVI_3'] < 0.2, 
    0, 
    gdf0['NDVI_3'])

#4
gdf0['NDVI_4'] = ndf4['NDVI_4']
gdf0['NDVI_4'] = np.where(
    gdf0['NDVI_4'] < 0.2, 
    0, 
    gdf0['NDVI_4'])

#5
gdf0['NDVI_5'] = ndf5['NDVI_5']
gdf0['NDVI_5'] = np.where(
    gdf0['NDVI_5'] < 0.2, 
    0, 
    gdf0['NDVI_5'])

#6
gdf0['NDVI_6'] = ndf6['NDVI_6']
gdf0['NDVI_6'] = np.where(
    gdf0['NDVI_6'] < 0.2, 
    0, 
    gdf0['NDVI_6'])

#7
gdf0['NDVI_7'] = ndf7['NDVI_7']
gdf0['NDVI_7'] = np.where(
    gdf0['NDVI_7'] < 0.2, 
    0, 
    gdf0['NDVI_7'])

#8
gdf0['NDVI_8'] = ndf8['NDVI_8']
gdf0['NDVI_8'] = np.where(
    gdf0['NDVI_8'] < 0.2, 
    0, 
    gdf0['NDVI_8'])

gdf0.head()

,OBJECTID,Lat,Long,GenusSpeci,subclass,geometry,NDVI,NDVI_1,NDVI_2,NDVI_3,NDVI_4,NDVI_5,NDVI_6,NDVI_7,NDVI_8


In [13]:
b=gdf0[gdf0['NDVI_1']== 0].index
gdf0=gdf0.drop(b)

c=gdf0[gdf0['NDVI_2']== 0].index
gdf0=gdf0.drop(c)

d=gdf0[gdf0['NDVI_3']== 0].index
gdf0=gdf0.drop(d)

e=gdf0[gdf0['NDVI_4']== 0].index
gdf0=gdf0.drop(e)

f=gdf0[gdf0['NDVI_5']== 0].index
gdf0=gdf0.drop(f)

g=gdf0[gdf0['NDVI_6']== 0].index
gdf0=gdf0.drop(g)

h=gdf0[gdf0['NDVI_7']== 0].index
gdf0=gdf0.drop(h)

i=gdf0[gdf0['NDVI_8']== 0].index
gdf0=gdf0.drop(i)

gdf0.head()
print(len(gdf0.index))

0


In [14]:
# SBI
coord_list0 = [(x,y) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
print(coord_list0[0:5])
coord_list1= [(x-0.00000599299783809013749338974087784241,y+0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list2= [(x,y+0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list3= [(x+0.00000599299783809013749338974087784241,y+0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list4= [(x+0.00000599299783809013749338974087784241,y) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list5= [(x+0.00000599299783809013749338974087784241,y-0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list6= [(x,y-0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list7= [(x-0.00000599299783809013749338974087784241,y-0.0000059299783809007439469768700121737) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]
coord_list8= [(x-0.00000599299783809013749338974087784241,y) for x,y in zip(gdf0['geometry'].x , gdf0['geometry'].y)]

[]


In [15]:
sdf1=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
sdf2=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
sdf3=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
sdf4=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
sdf5=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
sdf6=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
sdf7=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
sdf8=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)

In [16]:
file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/SBI.tif'
src = rasterio.open(file) # 파일 읽기
gdf0[Path(file).stem] = [x for x in src.sample(coord_list0)] # 포인트 샘플링
gdf0[Path(file).stem] = gdf0[Path(file).stem].astype('float64')
print(len(sdf1.index))
#1
sdf1[Path(file).stem] = [x for x in src.sample(coord_list1)] # 포인트 샘플링
sdf1[Path(file).stem] = sdf1[Path(file).stem].astype('float64')
sdf1.rename(columns={'SBI': 'SBI_1'}, inplace=True)
sdf1.head()
print(len(sdf1.index))

#2
sdf2[Path(file).stem] = [x for x in src.sample(coord_list2)] # 포인트 샘플링
sdf2[Path(file).stem] = sdf2[Path(file).stem].astype('float64')
sdf2.rename(columns={'SBI': 'SBI_2'}, inplace=True)
sdf2.head()
print(len(sdf2.index))

#3
sdf3[Path(file).stem] = [x for x in src.sample(coord_list3)] # 포인트 샘플링
sdf3[Path(file).stem] = sdf3[Path(file).stem].astype('float64')
sdf3.rename(columns={'SBI': 'SBI_3'}, inplace=True)
sdf3.head()
print(len(sdf3.index))

#4
sdf4[Path(file).stem] = [x for x in src.sample(coord_list4)] # 포인트 샘플링
sdf4[Path(file).stem] = sdf4[Path(file).stem].astype('float64')
sdf4.rename(columns={'SBI': 'SBI_4'}, inplace=True)
sdf4.head()
print(len(sdf4.index))

#5
sdf5[Path(file).stem] = [x for x in src.sample(coord_list5)] # 포인트 샘플링
sdf5[Path(file).stem] = sdf5[Path(file).stem].astype('float64')
sdf5.rename(columns={'SBI': 'SBI_5'}, inplace=True)
sdf5.head()
print(len(sdf5.index))

#6
sdf6[Path(file).stem] = [x for x in src.sample(coord_list6)] # 포인트 샘플링
sdf6[Path(file).stem] = sdf6[Path(file).stem].astype('float64')
sdf6.rename(columns={'SBI': 'SBI_6'}, inplace=True)
sdf6.head()
print(len(sdf6.index))

#7
sdf7[Path(file).stem] = [x for x in src.sample(coord_list7)] # 포인트 샘플링
sdf7[Path(file).stem] = sdf7[Path(file).stem].astype('float64')
sdf7.rename(columns={'SBI': 'SBI_7'}, inplace=True)
sdf7.head()
print(len(sdf7.index))

#8
sdf8[Path(file).stem] = [x for x in src.sample(coord_list8)] # 포인트 샘플링
sdf8[Path(file).stem] = sdf8[Path(file).stem].astype('float64')
sdf8.rename(columns={'SBI': 'SBI_8'}, inplace=True)
sdf8.head()
print(len(sdf8.index))

0
0
0
0
0
0
0
0
0


In [17]:
#1
gdf0['SBI_1'] = sdf1['SBI_1']
#2
gdf0['SBI_2'] = sdf2['SBI_2']
#3
gdf0['SBI_3'] = sdf3['SBI_3']
#4
gdf0['SBI_4'] = sdf4['SBI_4']
#5
gdf0['SBI_5'] = sdf5['SBI_5']
#6
gdf0['SBI_6'] = sdf6['SBI_6']
#7
gdf0['SBI_7'] = sdf7['SBI_7']
#8
gdf0['SBI_8'] = sdf8['SBI_8']
gdf0.head()

,OBJECTID,Lat,Long,GenusSpeci,subclass,geometry,NDVI,NDVI_1,NDVI_2,NDVI_3,...,NDVI_8,SBI,SBI_1,SBI_2,SBI_3,SBI_4,SBI_5,SBI_6,SBI_7,SBI_8


In [18]:
ydf1=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ydf2=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ydf3=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ydf4=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ydf5=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ydf6=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ydf7=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
ydf8=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)

In [19]:
from pathlib import Path # pathlib
file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/YVI.tif'
src = rasterio.open(file) # 파일 읽기
gdf0[Path(file).stem] = [x for x in src.sample(coord_list0)] # 포인트 샘플링
gdf0[Path(file).stem] = gdf0[Path(file).stem].astype('float64')

#1
ydf1[Path(file).stem] = [x for x in src.sample(coord_list1)] # 포인트 샘플링
ydf1[Path(file).stem] = ydf1[Path(file).stem].astype('float64')
ydf1.rename(columns={'YVI': 'YVI_1'}, inplace=True)
ydf1.head()
print(len(ydf1.index))

#2
ydf2[Path(file).stem] = [x for x in src.sample(coord_list2)] # 포인트 샘플링
ydf2[Path(file).stem] = ydf2[Path(file).stem].astype('float64')
ydf2.rename(columns={'YVI': 'YVI_2'}, inplace=True)
ydf2.head()
print(len(ydf2.index))

#3
ydf3[Path(file).stem] = [x for x in src.sample(coord_list3)] # 포인트 샘플링
ydf3[Path(file).stem] = ydf3[Path(file).stem].astype('float64')
ydf3.rename(columns={'YVI': 'YVI_3'}, inplace=True)
ydf3.head()
print(len(ydf3.index))

#4
ydf4[Path(file).stem] = [x for x in src.sample(coord_list4)] # 포인트 샘플링
ydf4[Path(file).stem] = ydf4[Path(file).stem].astype('float64')
ydf4.rename(columns={'YVI': 'YVI_4'}, inplace=True)
ydf4.head()
print(len(ydf4.index))

#5
ydf5[Path(file).stem] = [x for x in src.sample(coord_list5)] # 포인트 샘플링
ydf5[Path(file).stem] = ydf5[Path(file).stem].astype('float64')
ydf5.rename(columns={'YVI': 'YVI_5'}, inplace=True)
ydf5.head()
print(len(ydf5.index))

#6
ydf6[Path(file).stem] = [x for x in src.sample(coord_list6)] # 포인트 샘플링
ydf6[Path(file).stem] = ydf6[Path(file).stem].astype('float64')
ydf6.rename(columns={'YVI': 'YVI_6'}, inplace=True)
ydf6.head()
print(len(ydf6.index))

#7
ydf7[Path(file).stem] = [x for x in src.sample(coord_list7)] # 포인트 샘플링
ydf7[Path(file).stem] = ydf7[Path(file).stem].astype('float64')
ydf7.rename(columns={'YVI': 'YVI_7'}, inplace=True)
ydf7.head()
print(len(ydf7.index))

#8
ydf8[Path(file).stem] = [x for x in src.sample(coord_list8)] # 포인트 샘플링
ydf8[Path(file).stem] = ydf8[Path(file).stem].astype('float64')
ydf8.rename(columns={'YVI': 'YVI_8'}, inplace=True)
ydf8.head()
print(len(ydf8.index))

0
0
0
0
0
0
0
0


In [20]:
#1
gdf0['YVI_1'] = ydf1['YVI_1']
#2
gdf0['YVI_2'] = ydf2['YVI_2']
#3
gdf0['YVI_3'] = ydf3['YVI_3']
#4
gdf0['YVI_4'] = ydf4['YVI_4']
#5
gdf0['YVI_5'] = ydf5['YVI_5']
#6
gdf0['YVI_6'] = ydf6['YVI_6']
#7
gdf0['YVI_7'] = ydf7['YVI_7']
#8
gdf0['YVI_8'] = ydf8['YVI_8']
gdf0.head()

,OBJECTID,Lat,Long,GenusSpeci,subclass,geometry,NDVI,NDVI_1,NDVI_2,NDVI_3,...,SBI_8,YVI,YVI_1,YVI_2,YVI_3,YVI_4,YVI_5,YVI_6,YVI_7,YVI_8


In [21]:
# GVI
gdf1=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
gdf2=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
gdf3=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
gdf4=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
gdf5=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
gdf6=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
gdf7=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
gdf8=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)

In [22]:
file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/GVI.tif'
src = rasterio.open(file) # 파일 읽기
gdf0[Path(file).stem] = [x for x in src.sample(coord_list0)] # 포인트 샘플링
gdf0[Path(file).stem] = gdf0[Path(file).stem].astype('float64')

#1
gdf1[Path(file).stem] = [x for x in src.sample(coord_list1)] # 포인트 샘플링
gdf1[Path(file).stem] = gdf1[Path(file).stem].astype('float64')
gdf1.rename(columns={'GVI': 'GVI_1'}, inplace=True)
gdf1.head()
print(len(gdf1.index))

#2
gdf2[Path(file).stem] = [x for x in src.sample(coord_list2)] # 포인트 샘플링
gdf2[Path(file).stem] = gdf2[Path(file).stem].astype('float64')
gdf2.rename(columns={'GVI': 'GVI_2'}, inplace=True)
gdf2.head()
print(len(gdf2.index))

#3
gdf3[Path(file).stem] = [x for x in src.sample(coord_list3)] # 포인트 샘플링
gdf3[Path(file).stem] = gdf3[Path(file).stem].astype('float64')
gdf3.rename(columns={'GVI': 'GVI_3'}, inplace=True)
gdf3.head()
print(len(gdf3.index))

#4
gdf4[Path(file).stem] = [x for x in src.sample(coord_list4)] # 포인트 샘플링
gdf4[Path(file).stem] = gdf4[Path(file).stem].astype('float64')
gdf4.rename(columns={'GVI': 'GVI_4'}, inplace=True)
gdf4.head()
print(len(gdf4.index))

#5
gdf5[Path(file).stem] = [x for x in src.sample(coord_list5)] # 포인트 샘플링
gdf5[Path(file).stem] = gdf5[Path(file).stem].astype('float64')
gdf5.rename(columns={'GVI': 'GVI_5'}, inplace=True)
gdf5.head()
print(len(gdf5.index))

#6
gdf6[Path(file).stem] = [x for x in src.sample(coord_list6)] # 포인트 샘플링
gdf6[Path(file).stem] = gdf6[Path(file).stem].astype('float64')
gdf6.rename(columns={'GVI': 'GVI_6'}, inplace=True)
gdf6.head()
print(len(gdf6.index))

#7
gdf7[Path(file).stem] = [x for x in src.sample(coord_list7)] # 포인트 샘플링
gdf7[Path(file).stem] = gdf7[Path(file).stem].astype('float64')
gdf7.rename(columns={'GVI': 'GVI_7'}, inplace=True)
gdf7.head()
print(len(gdf7.index))

#8
gdf8[Path(file).stem] = [x for x in src.sample(coord_list8)] # 포인트 샘플링
gdf8[Path(file).stem] = gdf8[Path(file).stem].astype('float64')
gdf8.rename(columns={'GVI': 'GVI_8'}, inplace=True)
gdf8.head()
print(len(gdf8.index))

0
0
0
0
0
0
0
0


In [23]:
#1
gdf0['GVI_1'] = gdf1['GVI_1']
#2
gdf0['GVI_2'] = gdf2['GVI_2']
#3
gdf0['GVI_3'] = gdf3['GVI_3']
#4
gdf0['GVI_4'] = gdf4['GVI_4']
#5
gdf0['GVI_5'] = gdf5['GVI_5']
#6
gdf0['GVI_6'] = gdf6['GVI_6']
#7
gdf0['GVI_7'] = gdf7['GVI_7']
#8
gdf0['GVI_8'] = gdf8['GVI_8']
gdf0.head()

,OBJECTID,Lat,Long,GenusSpeci,subclass,geometry,NDVI,NDVI_1,NDVI_2,NDVI_3,...,YVI_8,GVI,GVI_1,GVI_2,GVI_3,GVI_4,GVI_5,GVI_6,GVI_7,GVI_8


In [24]:
#WBI
wdf1=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
wdf2=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
wdf3=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
wdf4=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
wdf5=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
wdf6=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
wdf7=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)
wdf8=gdf0.drop(["Lat","Long","GenusSpeci","subclass","geometry"],axis=1)

In [25]:
file = '../KOMPSAT 위성영상/데이터 전처리/4. data/br/WBI.tif'
src = rasterio.open(file) # 파일 읽기
gdf0[Path(file).stem] = [x for x in src.sample(coord_list0)] # 포인트 샘플링
gdf0[Path(file).stem] = gdf0[Path(file).stem].astype('float64')

#1
wdf1[Path(file).stem] = [x for x in src.sample(coord_list1)] # 포인트 샘플링
wdf1[Path(file).stem] = wdf1[Path(file).stem].astype('float64')
wdf1.rename(columns={'WBI': 'WBI_1'}, inplace=True)
wdf1.head()
print(len(wdf1.index))

#2
wdf2[Path(file).stem] = [x for x in src.sample(coord_list2)] # 포인트 샘플링
wdf2[Path(file).stem] = wdf2[Path(file).stem].astype('float64')
wdf2.rename(columns={'WBI': 'WBI_2'}, inplace=True)
wdf2.head()
print(len(wdf2.index))

#3
wdf3[Path(file).stem] = [x for x in src.sample(coord_list3)] # 포인트 샘플링
wdf3[Path(file).stem] = wdf3[Path(file).stem].astype('float64')
wdf3.rename(columns={'WBI': 'WBI_3'}, inplace=True)
wdf3.head()
print(len(wdf3.index))

#4
wdf4[Path(file).stem] = [x for x in src.sample(coord_list4)] # 포인트 샘플링
wdf4[Path(file).stem] = wdf4[Path(file).stem].astype('float64')
wdf4.rename(columns={'WBI': 'WBI_4'}, inplace=True)
wdf4.head()
print(len(wdf4.index))

#5
wdf5[Path(file).stem] = [x for x in src.sample(coord_list5)] # 포인트 샘플링
wdf5[Path(file).stem] = wdf5[Path(file).stem].astype('float64')
wdf5.rename(columns={'WBI': 'WBI_5'}, inplace=True)
wdf5.head()
print(len(wdf5.index))

#6
wdf6[Path(file).stem] = [x for x in src.sample(coord_list6)] # 포인트 샘플링
wdf6[Path(file).stem] = wdf6[Path(file).stem].astype('float64')
wdf6.rename(columns={'WBI': 'WBI_6'}, inplace=True)
wdf6.head()
print(len(wdf6.index))

#7
wdf7[Path(file).stem] = [x for x in src.sample(coord_list7)] # 포인트 샘플링
wdf7[Path(file).stem] = wdf7[Path(file).stem].astype('float64')
wdf7.rename(columns={'WBI': 'WBI_7'}, inplace=True)
wdf7.head()
print(len(wdf7.index))

#8
wdf8[Path(file).stem] = [x for x in src.sample(coord_list8)] # 포인트 샘플링
wdf8[Path(file).stem] = wdf8[Path(file).stem].astype('float64')
wdf8.rename(columns={'WBI': 'WBI_8'}, inplace=True)
wdf8.head()
print(len(wdf8.index))

0
0
0
0
0
0
0
0


In [26]:
#1
gdf0['WBI_1'] = wdf1['WBI_1']
#2
gdf0['WBI_2'] = wdf2['WBI_2']
#3
gdf0['WBI_3'] = wdf3['WBI_3']
#4
gdf0['WBI_4'] = wdf4['WBI_4']
#5
gdf0['WBI_5'] = wdf5['WBI_5']
#6
gdf0['WBI_6'] = wdf6['WBI_6']
#7
gdf0['WBI_7'] = wdf7['WBI_7']
#8
gdf0['WBI_8'] = wdf8['WBI_8']
gdf0.head()

,OBJECTID,Lat,Long,GenusSpeci,subclass,geometry,NDVI,NDVI_1,NDVI_2,NDVI_3,...,GVI_8,WBI,WBI_1,WBI_2,WBI_3,WBI_4,WBI_5,WBI_6,WBI_7,WBI_8


In [27]:
# gdf0 = gdf0.drop(['Lat', 'Long', 'GenusSpeci', 'geometry'], axis = 1)
# gdf0

In [28]:
gdf0 = gdf0.drop(['GenusSpeci'], axis = 1)
gdf0

,OBJECTID,Lat,Long,subclass,geometry,NDVI,NDVI_1,NDVI_2,NDVI_3,NDVI_4,...,GVI_8,WBI,WBI_1,WBI_2,WBI_3,WBI_4,WBI_5,WBI_6,WBI_7,WBI_8


In [116]:
br2 = gdf0.to_csv('br2_training.csv', index=False)
# br2 = pd.read_csv('manup2017_training.csv')
# up.head()